### Author: Rodolfo Elenes

Date Created: 8/5/2025

Change log:
8/5/2025 - Initialized

##### Imports

In [2]:
import pandas as pd
import duckdb
import datetime
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [3]:
%run ./common_utils.ipynb

Importing following functions: save_df(), construct_df_roster(), construct_df_teams(), concatenate_all_files()


# Get Runningbacks

In [4]:
con_memory = duckdb.connect(database=':memory:')
players = construct_df_roster(con_memory)
display(players)

,Team,ABV,Season,Position,Player,Birth_date,Height,Weight,pfr_id,Years_exp
0,Atlanta Falcons,ATL,1970,RB,Art Malone,1948-03-20,71.0,211.0,None,NaN
1,Atlanta Falcons,ATL,1970,RB,Cannonball Butler,1943-05-04,69.0,194.0,None,NaN
2,Atlanta Falcons,ATL,1970,RB,Harmon Wages,1946-05-18,73.0,215.0,None,NaN
3,Atlanta Falcons,ATL,1970,RB,Mike Brunson,1947-07-30,73.0,190.0,None,NaN
4,Atlanta Falcons,ATL,1970,RB,Paul Gipson,1946-03-21,72.0,210.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...
12439,Washington Redskins,WAS,2024,RB,Colson Yankoff,3/1/2000,76.0,225.0,YankCo00,0.0
12440,Washington Redskins,WAS,2024,RB,Demetric Felton,7/16/1998,69.0,185.0,FeltDe00,3.0
12441,Washington Redskins,WAS,2024,RB,Jeremy McNichols,12/26/1995,69.0,205.0,McNiJe00,7.0
12442,Washington Redskins,WAS,2024,RB,Kazmeir Allen,2/29/2000,68.0,175.0,None,1.0


In [5]:
# # Setup the starting dataframe

# players = pd.read_csv("../src/rosters/roster_2024.csv")
# df = players[(players['depth_chart_position'] == 'RB')].dropna(subset=['birth_date', 'pfr_id'])

# custom_add = pd.read_csv("../src/rosters/roster_2009.csv")
# custom_add = custom_add[(custom_add['full_name'] == "Ricky Williams")].fillna({'birth_date': '01/01/1990', 'pfr_id': 'UNKNOWN'})
# custom_add['depth_chart_position'] = 'RB'
# df = pd.concat([df, custom_add])

# display(df)

In [6]:
# Setup the starting dataframe

con_memory = duckdb.connect(database=':memory:')
df_roster = construct_df_roster(con_memory)
df = df_roster[(df_roster['Position'] == 'RB')].sort_values(by=['Player', 'Season']).reset_index(drop=True)

# Take slice of 1990 roster
df_temp = con_memory.execute("SELECT * FROM df").fetchdf()
display(df_temp)

# Take players, seasons, years_exp

# Filter to keep NaN ones

print(len(df.Player.unique().tolist()), 'Players')
print(df[(df.pfr_id.isnull())].shape[0], 'nulls')

df['pfr_id'] = df.groupby('Player')['pfr_id'].transform(lambda x: x.ffill().bfill())
df = df[['Player', 'pfr_id']].dropna().drop_duplicates('pfr_id').reset_index(drop=True)
df = df_roster[(df_roster.Player.isin(df.Player.unique().tolist())) 
               & (df_roster.pfr_id.isin(df.pfr_id.unique().tolist()))].reset_index(drop=True)

print(len(df.Player.unique().tolist()), 'Players')
print(df[(df.pfr_id.isnull())].shape[0], 'nulls')


display(df)
con_memory.close()

,Team,ABV,Season,Position,Player,Birth_date,Height,Weight,pfr_id,Years_exp
0,New York Jets,NYJ,1989,RB,A.B. Brown,1965-12-04,69.0,210.0,None,NaN
1,New York Jets,NYJ,1990,RB,A.B. Brown,1965-12-04,69.0,210.0,None,NaN
2,New York Jets,NYJ,1991,RB,A.B. Brown,1965-12-04,69.0,210.0,None,NaN
3,New York Jets,NYJ,1992,RB,A.B. Brown,1965-12-04,69.0,210.0,None,NaN
4,Green Bay Packers,GNB,2020,RB,A.J. Dillon,5/2/1998,72.0,250.0,DillAJ00,0.0
...,...,...,...,...,...,...,...,...,...,...
12439,Los Angeles Raiders,RAI,1987,RB,Zeph Lee,1963-06-17,75.0,215.0,None,NaN
12440,Los Angeles Raiders,RAI,1988,RB,Zeph Lee,1963-06-17,75.0,215.0,None,NaN
12441,New York Jets,NYJ,2022,RB,Zonovan Knight,4/11/2001,71.0,210.0,None,0.0
12442,Indianapolis Colts,IND,2014,RB,Zurlon Tipton,4/27/1989,72.0,231.0,None,0.0


2739 Players
9750 nulls
464 Players
0 nulls


,Team,ABV,Season,Position,Player,Birth_date,Height,Weight,pfr_id,Years_exp
0,Jacksonville Jaguars,JAX,1998,RB,Fred Taylor,1976-01-27,73.0,231.0,TaylFr00,NaN
1,Jacksonville Jaguars,JAX,1999,RB,Fred Taylor,1976-01-27,73.0,227.0,TaylFr00,NaN
2,New Orleans Saints,NOR,1999,RB,Ricky Williams,1977-05-21,70.0,236.0,WillRi00,NaN
3,Arizona Cardinals,ARI,2000,RB,Thomas Jones,1978-08-19,70.0,211.0,JoneTh00,NaN
4,Buffalo Bills,BUF,2000,RB,Sammy Morris,1977-03-23,72.0,228.0,MorrSa00,NaN
...,...,...,...,...,...,...,...,...,...,...
2674,Washington Redskins,WAS,2024,RB,Austin Ekeler,5/17/1995,70.0,200.0,EkelAu00,7.0
2675,Washington Redskins,WAS,2024,RB,Brian Robinson,3/22/1999,73.0,228.0,RobiBr01,2.0
2676,Washington Redskins,WAS,2024,RB,Colson Yankoff,3/1/2000,76.0,225.0,YankCo00,0.0
2677,Washington Redskins,WAS,2024,RB,Demetric Felton,7/16/1998,69.0,185.0,FeltDe00,3.0


# Exploratory Data Work

In [16]:
display(df.head()) #visualize the table
print(df.info()) #check data structure
print("\n")
print(df.isnull().any()) #check for nulls

#check duplicate rows
if df.duplicated().any():
    print('\nDuplicates found.')
else:
    print('\nNo duplicates found.')

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,...,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
82,2024,PIT,RB,RB,84.0,ACT,Cordarrelle Patterson,Cordarrelle,Patterson,3/17/1991,...,WC,A01,Cordarrelle,PAT387075,39975.0,32005041-5438-7075-4e25-a1072ce2f5ac,2013,2013.0,MIN,29.0
151,2024,MIA,RB,RB,31.0,ACT,Raheem Mostert,Dominique,Mostert,4/9/1992,...,REG,A01,Raheem,MOS822492,42718.0,32004d4f-5382-2492-b0f0-aec08c3525cf,2015,2015.0,NaN,NaN
176,2024,LV,RB,RB,8.0,INA,Ameer Abdullah,Ameer,Abdullah,6/13/1993,...,REG,A01,Ameer,ABD647726,42397.0,32004142-4464-7726-b649-e09413abd27c,2015,2015.0,DET,54.0
251,2024,BAL,RB,RB,22.0,ACT,Derrick Henry,Derrick,Henry,1/4/1994,...,DIV,A01,Derrick,HEN602843,43334.0,32004845-4e60-2843-9b64-3cd8fac319c3,2016,2016.0,TEN,45.0
278,2024,LAC,RB,RB,24.0,DEV,Ezekiel Elliott,Ezekiel,Elliott,7/22/1995,...,WC,P07,Ezekiel,ELL289284,43293.0,3200454c-4c28-9284-cdcc-74694c8d1749,2016,2016.0,DAL,4.0


<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, 82 to 268
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   season                   177 non-null    int64  
 1   team                     177 non-null    object 
 2   position                 177 non-null    object 
 3   depth_chart_position     177 non-null    object 
 4   jersey_number            177 non-null    float64
 5   status                   177 non-null    object 
 6   full_name                177 non-null    object 
 7   first_name               177 non-null    object 
 8   last_name                177 non-null    object 
 9   birth_date               177 non-null    object 
 10  height                   177 non-null    float64
 11  weight                   177 non-null    float64
 12  college                  177 non-null    object 
 13  gsis_id                  177 non-null    object 
 14  espn_id                  172 n

In [17]:
# check experience in league splits

df_numeric = df.select_dtypes(include=['number'])
df_numeric = df_numeric['years_exp'].value_counts().reset_index()
df_numeric.columns = ['years_exp', 'count']
display(df_numeric.sort_values("years_exp"))

,years_exp,count
2,0,28
1,1,30
0,2,31
4,3,21
3,4,22
6,5,13
7,6,11
5,7,16
9,8,2
8,9,2


# Create and Export Final Table

In [42]:
#Create age column and drop unneccesary columns

# df['birth_date'] = pd.to_datetime(df['birth_date'])
# opening_day = datetime.datetime(2024, 9, 5, 0, 0, 0) # September 5, 2024, 12:00 AM, opening day
# df['age'] = ((opening_day - df['birth_date']) / pd.Timedelta(days=365)).astype(int)
df['age'] = 100
df['gm_log_rtrvd'] = 0
df = df[['season', 'team', 'position', 'full_name', 'height', 'weight', 'age', 'years_exp', 'pfr_id', 'gm_log_rtrvd']].reset_index(drop=True)
display(df)

,season,team,position,full_name,height,weight,age,years_exp,pfr_id,gm_log_rtrvd
0,2024,PIT,RB,Cordarrelle Patterson,74.0,238,100,11,PattCo00,0
1,2024,MIA,RB,Raheem Mostert,70.0,197,100,9,MostRa00,0
2,2024,LV,RB,Ameer Abdullah,69.0,203,100,9,AbduAm00,0
3,2024,BAL,RB,Derrick Henry,75.0,247,100,8,HenrDe00,0
4,2024,LAC,RB,Ezekiel Elliott,72.0,228,100,8,ElliEz00,0
...,...,...,...,...,...,...,...,...,...,...
172,2024,GB,RB,MarShawn Lloyd,69.0,210,100,0,LloyMa00,0
173,2024,MIA,RB,Jaylen Wright,71.0,210,100,0,WrigJa04,0
174,2024,BUF,RB,Ray Davis,68.0,216,100,0,DaviRa04,0
175,2024,ARI,RB,Trey Benson,73.0,215,100,0,BensTr01,0


In [ ]:
# Export final table

df.to_csv('../tables/players_xref.csv', index = False)